In [1]:
from collections import namedtuple
import os
os.chdir("..")
os.getcwd()


'd:\\ineuron\\DL_project\\CNN_project'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path       #base untrained vgg16 archtecture model
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class CallbackPreparationConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    
  

In [3]:
from cnnProject.constant import *
from cnnProject.utils import read_yaml,create_directories

In [4]:
class ConfigrationManger:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifact_root])

    def get_callback_perparation_config(self) -> CallbackPreparationConfig :
        config = self.config.callback_preparation

        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(config.tensorboard_root_log_dir),
            Path(model_checkpoint_dir)
        ])

        callback_prepare_config = CallbackPreparationConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return callback_prepare_config
    
    
    def get_training_config(self) -> TrainingConfig :
        training = self.config.training
        prepare_base_model = self.config.base_model_preparation
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"PetImages")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs =  params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTAION,
            params_image_size = params.IMAGE_SIZE
            )

        return training_config


In [5]:
#CallbackPrepareComponent
import os
import time
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class CallbackPreparation:
    def __init__(self,config:CallbackPreparationConfig) -> None:
        self.config = config

    @property
    def _create_tb_callbacks(self):
        #good ideal to create a folder structure with timestamps
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")

        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True  #save only best weights aka loss was less
        )



    def get_tb_checkpoint_callback(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

   
    

In [6]:
#training Component
import os
import time
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class Training:
    def __init__(self,config:TrainingConfig) -> None:
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        #will be doing augmentation 
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],  #only takes rows and columns, channels not aka index 0,1
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear",
            # interpolation is a process of determining the unknown values that lie in between the known data points.
            # ex find value between 1 and 2 on a line. 
        )


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        #validation data
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset= "validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        #training data
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path,model: tf.keras.Model):
        model.save(path)


    #start training
    def train(self , callback_list):
        self.step_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.step_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

    


In [7]:
try:
    config = ConfigrationManger()
    callback_preparation_config = config.get_callback_perparation_config()
    callback_prepare = CallbackPreparation(config=callback_preparation_config)
    callback_lst = callback_prepare.get_tb_checkpoint_callback()



    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()  #create a training and validation generator similar to train_test_split but not exactly  same 
    training.train(
        callback_list = callback_lst,
    )
except Exception as e:
    raise e

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
 72/312 [=====>........................] - ETA: 31:20 - loss: 15.1567 - accuracy: 0.5080

d:\ineuron\DL_project\CNN_project\env\lib\site-packages\PIL\TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


 90/312 [=======>......................] - ETA: 30:11 - loss: 14.7849 - accuracy: 0.5102

: 

: 

In [ ]:
#!pip install scipy

You should consider upgrading via the 'c:\python\python394\python.exe -m pip install --upgrade pip' command.


In [ ]:
#passing data as key value pair
def example(x,**kwargs):
    print(locals())


extra = dict(y=34,z=343)
example(x=2,**extra)

{'x': 2, 'kwargs': {'y': 34, 'z': 343}}


In [17]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

False

In [24]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [26]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7617734512317975280
xla_global_id: -1
]


In [1]:
import tensorflow as tf
print(tf.reduce_sum(tf.random.normal([1000, 1000])))


tf.Tensor(-191.95245, shape=(), dtype=float32)


In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(physical_devices))



[]


NameError: name 'physical_devices' is not defined

In [4]:
!pip install --upgrade pip


  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.2.3
    Uninstalling pip-20.2.3:
      Successfully uninstalled pip-20.2.3


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\kanan\\AppData\\Local\\Temp\\pip-uninstall-xmfuie23\\pip.exe'
Consider using the `--user` option or check the permissions.



In [5]:
!pip install "tensorflow<2.11" 


     -------------------------------------- 455.9/455.9 MB 2.2 MB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     -------------------------------------- 438.7/438.7 kB 5.5 MB/s eta 0:00:00
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
     ---------------------------------------- 1.7/1.7 MB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 5.7 MB/s eta 0:00:00
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ---------------------------------------- 23.2/23.2 MB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 5.9/5.9 MB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 6.0 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 7.3 MB/s eta

In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]
